<a href="https://colab.research.google.com/github/tejomaygadgil/cgpos/blob/nn/notebooks/colab/1_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install wandb
import wandb
wandb.login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 22.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.8 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=1b711ca76bbd8d01193b2003c0cf710b7a05cbbedf35ccdaa22697cbc5ff5e1c
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
!git clone -b nn https://github.com/tejomaygadgil/cgpos.git
!pip install git+https://github.com/jtauber/greek-accentuation.git beta-code

Cloning into 'cgpos'...
remote: Enumerating objects: 2264, done.
remote: Counting objects: 100% (242/242), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 2264 (delta 159), reused 162 (delta 90), pack-reused 2022
Receiving objects: 100% (2264/2264), 101.88 MiB | 36.93 MiB/s, done.
Resolving deltas: 100% (1405/1405), done.
  Cloning https://github.com/jtauber/greek-accentuation.git to /tmp/pip-req-build-3mhajpfx
  Running command git clone --filter=blob:none --quiet https://github.com/jtauber/greek-accentuation.git /tmp/pip-req-build-3mhajpfx
  Resolved https://github.com/jtauber/greek-accentuation.git to commit 15ac5fd1cc82c8f9b91a4041f9b64399c9552097
  Preparing metadata (setup.py) ... done


In [13]:
%cd /content/cgpos
!make process_ft_data
!python src/pretrain.py setup pt_cloud

/content/cgpos
Initializing data directory
make: Nothing to be done for 'process_ft_data'.
2023-10-08 16:40:42,820 - __main__ - INFO - Pre-training setup:
2023-10-08 16:40:47,388 - util - INFO - Imported /content/drive/MyDrive/Colab Notebooks/pt_syl.pkl
2023-10-08 16:40:58,993 - util - INFO - Exported data/train/pt_params.pkl
2023-10-08 16:40:58,999 - util - INFO - Exported data/train/pt_stoi.pkl
2023-10-08 16:40:59,003 - util - INFO - Exported data/train/pt_itos.pkl
2023-10-08 16:40:59,355 - util - INFO - Exported data/train/pt_train
2023-10-08 16:40:59,363 - util - INFO - Exported data/train/pt_val
2023-10-08 16:40:59,363 - __main__ - INFO - vocab_size: 17,016
2023-10-08 16:40:59,363 - __main__ - INFO - train_size: 0.98
2023-10-08 16:40:59,363 - __main__ - INFO - n_chunks: 500
2023-10-08 16:40:59,363 - __main__ - INFO - Train set: 34,631,855 obs
2023-10-08 16:40:59,363 - __main__ - INFO - Val set: 708,210 obs

                                                  
                       

In [29]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), 354 bytes | 354.00 KiB/s, done.
From https://github.com/tejomaygadgil/cgpos
   32f1fbb..dd63b67  nn         -> origin/nn
Updating 32f1fbb..dd63b67
Fast-forward
 src/util.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [14]:
%load_ext autoreload
%autoreload 2

In [16]:
from collections import defaultdict
import random

from tqdm import tqdm
import torch
import torch.nn as nn

from src.model import Transformer
import src.config as cfg
from src.util import read_pkl, encode, decode, get_batch, display_bar

In [17]:
params = read_pkl(cfg.pt_params)
wandb.init(project="ncgpos_ft", config=params)
for param, value in params.items():
    globals()[param] = value

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

wandb: Currently logged in as: tejomaygadgil. Use `wandb login --relogin` to force relogin


cuda


In [20]:
# Read encodings
stoi = read_pkl(cfg.pt_stoi)
itos = read_pkl(cfg.pt_itos)
# Read data
ft_syl = read_pkl(cfg.ft_syl)
ft_targets = read_pkl(cfg.ft_targets)
ft_targets_map = read_pkl(cfg.ft_targets_map)
assert len(ft_syl) == len(ft_targets)
# Process data
default_stoi = defaultdict(lambda: 0, stoi) # Set to "<UNK>" if OOV
tokens = []
labels = []
for i, word in enumerate(ft_syl):
    for token in encode(default_stoi, word):
        tokens.append(token)
        labels.append(ft_targets[i][0])

tokens = torch.tensor(tokens, dtype=torch.long)
labels = torch.tensor(labels, dtype=torch.long)
assert len(tokens) == len(labels)
print(list(zip(ft_syl, ft_targets))[:5])
print(list(zip(tokens.tolist(), labels.tolist()))[:10])
# Train val split
tokens_chunks = torch.split(tokens, len(tokens) // (n_chunks - 1))
labels_chunks = torch.split(labels, len(labels) // (n_chunks - 1))
l = [1] * int(n_chunks * train_size) + [0] * int(n_chunks * (1 - train_size))
random.shuffle(l)
train_tokens = torch.cat([tokens_chunks[i] for i, v in enumerate(l) if v])
train_labels = torch.cat([labels_chunks[i] for i, v in enumerate(l) if v])
val_tokens = torch.cat([tokens_chunks[i] for i, v in enumerate(l) if not v])
val_labels = torch.cat([labels_chunks[i] for i, v in enumerate(l) if not v])

print(f"train_size: {train_size}")
print(f"n_chunks: {n_chunks}")
print(f"Train size: {len(train_tokens):,} obs")
print(f"Val size: {len(val_tokens):,} obs")
display_bar(l)


[(['Θου', 'κυ', 'δί', 'δης'], [2, 0, 1, 0, 0, 0, 1, 1, 0]), (['Ἀ', 'θη', 'ναῖ', 'ος'], [2, 0, 1, 0, 0, 0, 1, 1, 0]), (['ξυ', 'νέ', 'γρα', 'ψε'], [5, 3, 1, 7, 1, 1, 0, 0, 0]), (['τὸν'], [1, 0, 1, 0, 0, 0, 1, 4, 0]), (['πό', 'λε', 'μον'], [2, 0, 1, 0, 0, 0, 1, 4, 0])]
[(1025, 2), (7409, 2), (4853, 2), (5001, 2), (15916, 2), (5970, 2), (8861, 2), (9665, 2), (9519, 5), (8752, 5)]
train_size: 0.98
n_chunks: 500
Train size: 1,095,328 obs
Val size: 22,390 obs

                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  

In [22]:
model = Transformer(
    vocab_size=17016,
    block_size=256,
    emb_size=512,
    n_layer=6,
    n_head=8,
    dropout=0.6,
    device=device
)
model.load_state_dict(torch.load(cfg.pt_wts, map_location=torch.device(device)))
# Omg!!
model.lm_head = nn.Linear(512, len(ft_targets_map[1][0]))

m = model.to(device)

In [23]:
@torch.no_grad()
def estimate_loss(eval_iters, device, *batch_args):
    out = []
    model.eval()
    for tokens, labels in [(train_tokens, train_labels), (val_tokens, val_labels)]:
        losses = torch.zeros(eval_iters, device=device)
        for k in range(eval_iters):
            X, Y = get_batch(tokens, *batch_args, y=labels)
            _, loss = model(X, Y)
            losses[k] = loss.item()
        out.append(losses.mean())
    model.train()
    return out

In [37]:
data = train_tokens


In [41]:
x.to(device)

RuntimeError: ignored

In [39]:
ix = torch.randint(len(data) - block_size, (batch_size,))
x = torch.stack([data[i : i + block_size] for i in ix])


In [ ]:
ix = torch.randint(len(data) - block_size, (batch_size,))
x = torch.stack([data[i : i + block_size] for i in ix])
x = x.to(device)
y = y.to(device)
return x


In [33]:
def get_batch(data, block_size, batch_size, device, y=None):
    # Generate a small batch of data of inputs x and targets y
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    if y is not None:
        y = torch.stack([y[i + 1 : i + block_size + 1] for i in ix])
    else:
        y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x = x.to(device)
    y = y.to(device)
    return x
    # return x, y

In [34]:
xb, yb = get_batch(train_tokens, block_size, batch_size, device, y=None)

RuntimeError: ignored

In [25]:
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)
for step in tqdm(range(max_iters)):
    # Evaluate training and val loss every eval_interval
    # if (step % eval_interval == 0) or (iter == max_iters - 1):
    #     train_loss, val_loss = estimate_loss(
    #         eval_iters, device, block_size, batch_size, device
    #     )
    #     wandb.log({"train_loss": train_loss, "val_loss": val_loss})
    #     with logging_redirect_tqdm():
    #         logging.info(
    #             f"step {step}: train loss {train_loss:.4f}, val loss {val_loss:.4f}"
    #         )
    #         logging.info(generate(generate_len, block_size, model, device))

    # Sample batch
    xb, yb = get_batch(train_tokens, block_size, batch_size, device, y=train_labels)
    _, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

wandb.finish()

  0%|          | 0/10000 [00:00<?, ?it/s]


RuntimeError: ignored